In [1]:
import colamatch as clm
from colamatch import constellation_matching as cm
import numpy as np
import cv2
from matplotlib import pyplot as plt
%matplotlib notebook

## Define filenames and read images and landmarks

In [2]:
s_fixed = 1
s_moving = 2
fixed_file = "../data/section_%s.tif"%(str(s_fixed).zfill(4))
moving_file = "../data/section_%s.tif"%(str(s_moving).zfill(4)) 
fixed_landmarks = "../data/section_%s.landmarks.txt"%(str(s_fixed).zfill(4))
moving_landmarks = "../data/section_%s.landmarks.txt"%(str(s_moving).zfill(4))

In [3]:
# read landmarks and images
l_fixed = np.loadtxt(fixed_landmarks).astype('int')
l_moving = np.loadtxt(moving_landmarks).astype('int')
img_fixed = cv2.imread(fixed_file,0)
img_moving = cv2.imread(moving_file,0)
print(len(l_fixed), len(l_moving))

(36, 46)


In [4]:
def all_4cliques(pts):
    """
    return 
    """
    N = pts.shape[0]
    cliques = np.mgrid[0:N,0:N,0:N,0:N].reshape((4,-1)).T
    return [pts[c,:] for c in cliques]

In [5]:
# build index for fixed image
import imp
imp.reload(cm)
c1 = clm.ExhaustiveSampler(min(len(l_fixed),len(l_moving)),4)
c2 = clm.RandomSampler(min(len(l_fixed),len(l_moving)),4, 1500)
matches = cm.match(l_fixed, l_moving, c2, radius=0.13, lamda=2, 
                   ransac=0.1)

mts vor RANSAC: 0


IndexError: too many indices for array

In [ ]:
print(len(matches))

In [ ]:
cmap = plt.cm.get_cmap("hsv", len(matches))
fig,axs = plt.subplots(1,2)
axs[0].imshow(img_fixed, cmap='gray')
axs[1].imshow(img_moving, cmap='gray')
axs[0].plot(l_fixed[:,0], l_fixed[:,1],'w+',ms=15)
axs[1].plot(l_moving[:,0], l_moving[:,1],'w+',ms=15)
axs[0].axis('off')
axs[1].axis('off')
for i,candidate in enumerate(matches): 
    axs[0].plot(candidate[0,0], candidate[0,1], c=cmap(i), marker='o')
    axs[1].plot(candidate[1,0], candidate[1,1], c=cmap(i), marker='o')
fig.show()